# Importar datos desde drive

In [6]:
from google.colab import drive
from getpass import getpass
import os

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
os.chdir('/content/drive/My Drive/')
token = getpass('Introduce tu token de GitHub: ')
username = "arantafall"
prj_name = "prediccion-ataques-ciber"

# Clonar el repositorio usando el token
os.system(f'git clone https://{token}@github.com/{username}/{prj_name}.git')

Introduce tu token de GitHub: ··········


32768

In [8]:
os.chdir(f'/content/drive/My Drive/{prj_name}')

In [9]:
%ls -A

'Copy of Predict_AJAX.ipynb'   datos_modificados.csv   Predict_AJAX.ipynb
 data/                         .git/                   README.md


In [10]:
!git config --global user.email "Aran.Tafall.campusimpelia@gmail.com"
!git config --global user.name "arantafall"

# KNN

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [75]:
df = pd.read_csv("./datos_modificados.csv")

In [77]:
df.columns = df.columns.str.strip()

In [78]:
print(df.head())
print(df.info())
print(df.describe())

   Destination Port  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0                80          38308                  1                       1   
1               389            479                 11                       5   
2                88           1095                 10                       6   
3               389          15206                 17                      12   
4                88           1092                  9                       6   

   Total Length of Fwd Packets  Total Length of Bwd Packets  \
0                            6                            6   
1                          172                          326   
2                         3150                         3150   
3                         3452                         6660   
4                         3150                         3152   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                      6                      6            

In [79]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [80]:
print(y.value_counts())

Label
BENIGN              439332
DoS Hulk            230124
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name: count, dtype: int64


In [81]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [83]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [84]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))
print("Reporte de Clasificación:\n", classification_report(y_test, target_names=le.classes_))


In [ ]:
from sklearn.model_selection import cross_val_score

error_rates = []
k_values = range(1, 21)  # Probar valores de K de 1 a 20

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5)  # 5-fold cross-validation
    error_rates.append(1 - scores.mean())

# Graficar el error para diferentes valores de K
plt.figure(figsize=(10, 5))
plt.plot(k_values, error_rates, marker='o', linestyle='dashed', color='blue')
plt.xlabel("K")
plt.ylabel("Error Rate")
plt.title("Eligiendo el mejor valor de K")
plt.show()

In [ ]:
k_optimo = 3  # Supongamos que el mejor K fue 7 en el análisis previo
knn_optimo = KNeighborsClassifier(n_neighbors=k_optimo)

knn_optimo.fit(X_train, y_train)  # Reentrenar con los datos de entrenamiento

In [ ]:
y_pred_optimo = knn_optimo.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred_optimo))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred_optimo))
print("Reporte de Clasificación:\n", classification_report(y_test, target_names=le.classes_))

In [ ]:
import joblib
joblib.dump(knn_optimo, "modelo_knn.pkl")  # Guardar modelo entrenado
joblib.dump(scaler, "scaler.pkl")  # Guardar el scaler usado

In [ ]:
knn_cargado = joblib.load("modelo_knn.pkl")
scaler_cargado = joblib.load("scaler.pkl")

In [ ]:
X_nuevo = [[valor1, valor2, ..., valor69]]  # Nuevos datos a predecir
X_nuevo = scaler_cargado.transform(X_nuevo)  # Normalizar igual que los datos originales
prediccion = knn_cargado.predict(X_nuevo)
print("Predicción:", le.inverse_transform(prediccion))  # Convertir de número a categoría

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(knn_optimo, X_test, y_test, cmap="Blues")
plt.title("Matriz de Confusión - KNN Óptimo")
plt.show()